In [22]:
import RPi.GPIO as GPIO
import time
import os
import datetime
import numpy as np
# GPIO.setwarnings(False)

In [29]:
# !sudo dtoverlay w1-gpio gpiopin=6 pullup=0 # header pin 7

In [36]:
def wireless_ini():
    GPIO.cleanup()
    # set the pins numbering mode
    GPIO.setmode(GPIO.BOARD)
    # Select the GPIO pins used for the encoder K0-K3 data inputs
    GPIO.setup(11, GPIO.OUT)
    GPIO.setup(15, GPIO.OUT)
    GPIO.setup(16, GPIO.OUT)
    GPIO.setup(13, GPIO.OUT)
    # Select the signal used to select ASK/FSK
    GPIO.setup(18, GPIO.OUT)
    # Select the signal used to enable/disable the modulator
    GPIO.setup(22, GPIO.OUT)
    # Disable the modulator by setting CE pin lo
    GPIO.output (22, False)
    # Set the modulator to ASK for On Off Keying
    # by setting MODSEL pin lo
    GPIO.output (18, False)
    # Initialise K0-K3 inputs of the encoder to 0000
    GPIO.output (11, False)
    GPIO.output (15, False)
    GPIO.output (16, False)
    GPIO.output (13, False)
    
def wireless_one():
    print('turning on radiator')
    GPIO.output (11, True)
    GPIO.output (15, True)
    GPIO.output (16, True)
    GPIO.output (13, True)
    # let it settle, encoder requires this
    time.sleep(0.1)
    # Enable the modulator
    GPIO.output (22, True)
    # keep enabled for a period
    time.sleep(0.25)
    # Disable the modulator
    GPIO.output (22, False)
    GPIO.cleanup()

In [39]:
def wireless_one_off():
    print ("sending code 0111 Socket 1 off")
    GPIO.output (11, True)
    GPIO.output (15, True)
    GPIO.output (16, True)
    GPIO.output (13, False)
    # let it settle, encoder requires this
    time.sleep(0.1)
    # Enable the modulator
    GPIO.output (22, True)
    # keep enabled for a period
    time.sleep(0.25)
    # Disable the modulator
    GPIO.output (22, False)
    GPIO.cleanup()

In [41]:
wireless_ini()
wireless_one()


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  


turning on radiator


In [42]:
wireless_ini()
wireless_one_off()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  


sending code 0111 Socket 1 off


In [49]:
def temp_ini():
    GPIO.cleanup()
    GPIO.setmode(GPIO.BCM)
    GPIO.setup(21, GPIO.OUT)
    GPIO.output(21,1)

    os.system('modprobe w1-gpio')
    os.system('modprobe w1-therm')

    temp_loc = '/sys/bus/w1/devices/'
#     os.chdir(temp_loc)
#     print(os.listdir())
    device = '28-0213924576ef/'
    os.chdir(temp_loc+device)
#     print(os.getcwd())
    
temp_ini()    

In [50]:
def temp_raw():
    temp_sensor = 'w1_slave'
    f=open(temp_sensor,'r')
    lines = f.readlines()
    f.close()
    return lines

def read_temp():
    temp_ini()
    lines=temp_raw()
    while lines[0].strip()[-3:]!='YES':
        time.sleep(0.2)
        lines = temp_raw()
    temp_output = lines[1].find('t=')
    if temp_output != -1:
        temp_string = lines[1].strip()[temp_output+2:]
        temp_c = float(temp_string)/1000.0
        return temp_c
    

In [52]:
# temp_ini()
read_temp()

20.625

In [ ]:
temps = []
trigger_block = False
block_time = datetime.datetime.now().time() #ini
t = datetime.datetime.now().time()
print('entering loop')
while t.hour<9:
    curr = read_temp()
    print(curr)
    temps = temps + [curr]
    t = datetime.datetime.now().time()
    if (curr <= 20.625)  & (t.hour < 9) & (trigger_block == False):
        trigger_block = True  
        wireless_ini()
        wireless_one()
    elif (curr>20.725)& (trigger_block == True):
        wireless_ini()
        wireless_one_off()
        trigger_block = False
    
    time.sleep(10)

entering loop
20.625
turning on radiator


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: No channels have been set up yet - nothing to clean up!  Try cleaning up at the end of your program instead!
  


20.437
20.5
20.875
sending code 0111 Socket 1 off
21.5
21.187
20.812
20.75
20.812
20.562
turning on radiator
20.625
20.625
20.375
20.5
20.5
20.437


In [21]:
t.hour

1

In [27]:
temps = []

In [29]:
temps = temps + [1]

In [30]:
temps

[1]